In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!pip install rouge

#importing all necessary libraries
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import reuters
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
import nltk.data
import math
import re
import nltk
from nltk.translate.bleu_score import sentence_bleu
from rouge import Rouge 
nltk.download('all')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping grammars/basque_grammars.zip.
[nltk_data]    | Downloading package bcp47 to /root/nltk_data...
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   U

True

In [ ]:
import math
from nltk import sent_tokenize, word_tokenize, PorterStemmer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer

In [ ]:
with open("/content/gdrive/MyDrive/project/story.txt") as f:
  text = f.read()

In [ ]:
sentences = sent_tokenize(text)
total_documents = len(sentences)
print(total_documents)
print(sentences)

69
['\nIN WHICH PHILEAS FOGG AND PASSEPARTOUT ACCEPT EACH OTHER,\nTHE ONE AS MASTER, THE OTHER AS MAN\n\n\nMr. Phileas Fogg lived, in 1872, at No.', '7, Saville Row, Burlington\nGardens, the house in which Sheridan died in 1814.', 'He was one of the\nmost noticeable members of the Reform Club, though he seemed always to\navoid attracting attention; an enigmatical personage, about whom little\nwas known, except that he was a polished man of the world.', 'People said\nthat he resembled Byron--at least that his head was Byronic; but he was\na bearded, tranquil Byron, who might live on a thousand years without\ngrowing old.', 'Certainly an Englishman, it was more doubtful whether Phileas Fogg was\na Londoner.', 'He was never seen on \'Change, nor at the Bank, nor in the\ncounting-rooms of the "City"; no ships ever came into London docks of\nwhich he was the owner; he had no public employment; he had never been\nentered at any of the Inns of Court, either at the Temple, or Lincoln\'s\nInn, 

In [ ]:
len(sentences)

69

In [ ]:
len(text)

8502

In [ ]:
def _create_frequency_matrix(sentences):
    frequency_matrix = {}
    stopWords = set(stopwords.words("english"))
    ps = PorterStemmer()

    for sent in sentences:
        # counts frequencies of words in every sentence
        freq_table = {}
        words = word_tokenize(sent)

        for word in words:
            #stopwords removal
            if word in stopWords or len(word) < 2 or not word.isalpha():
                continue
            # stemming and lowercasing
            word = ps.stem(word.lower())
            if word not in freq_table:
                freq_table[word] = 0
            freq_table[word] += 1
        
        #save frequencies as sent -> word_frequencies
        frequency_matrix[sent] = freq_table

    return frequency_matrix

In [ ]:
t = [word for word in word_tokenize(sentences[0]) if word not in stopwords.words("english") and word.isalpha()]
t

['IN',
 'WHICH',
 'PHILEAS',
 'FOGG',
 'AND',
 'PASSEPARTOUT',
 'ACCEPT',
 'EACH',
 'OTHER',
 'THE',
 'ONE',
 'AS',
 'MASTER',
 'THE',
 'OTHER',
 'AS',
 'MAN',
 'Phileas',
 'Fogg',
 'lived',
 'No']

In [ ]:
frequency_matrix = _create_frequency_matrix(sentences)
frequency_matrix


{'\nIN WHICH PHILEAS FOGG AND PASSEPARTOUT ACCEPT EACH OTHER,\nTHE ONE AS MASTER, THE OTHER AS MAN\n\n\nMr. Phileas Fogg lived, in 1872, at No.': {'in': 1,
  'which': 1,
  'philea': 2,
  'fogg': 2,
  'and': 1,
  'passepartout': 1,
  'accept': 1,
  'each': 1,
  'other': 2,
  'the': 2,
  'one': 1,
  'as': 2,
  'master': 1,
  'man': 1,
  'live': 1,
  'no': 1},
 '7, Saville Row, Burlington\nGardens, the house in which Sheridan died in 1814.': {'savil': 1,
  'row': 1,
  'burlington': 1,
  'garden': 1,
  'hous': 1,
  'sheridan': 1,
  'die': 1},
 'He was one of the\nmost noticeable members of the Reform Club, though he seemed always to\navoid attracting attention; an enigmatical personage, about whom little\nwas known, except that he was a polished man of the world.': {'he': 1,
  'one': 1,
  'notic': 1,
  'member': 1,
  'reform': 1,
  'club': 1,
  'though': 1,
  'seem': 1,
  'alway': 1,
  'avoid': 1,
  'attract': 1,
  'attent': 1,
  'enigmat': 1,
  'personag': 1,
  'littl': 1,
  'known': 1,
 

In [ ]:
def _create_tf_matrix(freq_matrix):
    tf_matrix = {}

    for sent, f_table in freq_matrix.items():
        tf_table = {}

        count_words_in_sentence = len(f_table)
        for word, count in f_table.items():
            tf_table[word] = count / count_words_in_sentence

        tf_matrix[sent] = tf_table

    return tf_matrix
tf_matrix = _create_tf_matrix(frequency_matrix)
tf_matrix

{'\nIN WHICH PHILEAS FOGG AND PASSEPARTOUT ACCEPT EACH OTHER,\nTHE ONE AS MASTER, THE OTHER AS MAN\n\n\nMr. Phileas Fogg lived, in 1872, at No.': {'in': 0.0625,
  'which': 0.0625,
  'philea': 0.125,
  'fogg': 0.125,
  'and': 0.0625,
  'passepartout': 0.0625,
  'accept': 0.0625,
  'each': 0.0625,
  'other': 0.125,
  'the': 0.125,
  'one': 0.0625,
  'as': 0.125,
  'master': 0.0625,
  'man': 0.0625,
  'live': 0.0625,
  'no': 0.0625},
 '7, Saville Row, Burlington\nGardens, the house in which Sheridan died in 1814.': {'savil': 0.14285714285714285,
  'row': 0.14285714285714285,
  'burlington': 0.14285714285714285,
  'garden': 0.14285714285714285,
  'hous': 0.14285714285714285,
  'sheridan': 0.14285714285714285,
  'die': 0.14285714285714285},
 'He was one of the\nmost noticeable members of the Reform Club, though he seemed always to\navoid attracting attention; an enigmatical personage, about whom little\nwas known, except that he was a polished man of the world.': {'he': 0.05,
  'one': 0.05,

In [ ]:
def _create_documents_per_words(freq_matrix):
    word_per_doc_table = {}

    for sent, f_table in freq_matrix.items():
        for word, count in f_table.items():
            if word in word_per_doc_table:
                word_per_doc_table[word] += 1
            else:
                word_per_doc_table[word] = 1

    return word_per_doc_table
sentences = sent_tokenize(text)
documents_per_words = _create_documents_per_words(frequency_matrix)
documents_per_words

{'in': 1,
 'which': 1,
 'philea': 12,
 'fogg': 17,
 'and': 1,
 'passepartout': 7,
 'accept': 1,
 'each': 1,
 'other': 1,
 'the': 6,
 'one': 5,
 'as': 1,
 'master': 2,
 'man': 3,
 'live': 5,
 'no': 2,
 'savil': 6,
 'row': 6,
 'burlington': 1,
 'garden': 1,
 'hous': 4,
 'sheridan': 1,
 'die': 1,
 'he': 16,
 'notic': 1,
 'member': 5,
 'reform': 4,
 'club': 5,
 'though': 2,
 'seem': 4,
 'alway': 3,
 'avoid': 1,
 'attract': 1,
 'attent': 1,
 'enigmat': 1,
 'personag': 1,
 'littl': 3,
 'known': 3,
 'except': 1,
 'polish': 1,
 'world': 2,
 'peopl': 2,
 'said': 3,
 'resembl': 1,
 'byron': 1,
 'least': 4,
 'head': 3,
 'beard': 1,
 'tranquil': 2,
 'might': 1,
 'thousand': 2,
 'year': 4,
 'without': 2,
 'grow': 1,
 'old': 1,
 'certainli': 3,
 'englishman': 1,
 'doubt': 1,
 'whether': 1,
 'london': 4,
 'never': 5,
 'seen': 2,
 'bank': 1,
 'citi': 1,
 'ship': 1,
 'ever': 2,
 'came': 1,
 'dock': 1,
 'owner': 1,
 'public': 1,
 'employ': 1,
 'enter': 1,
 'inn': 1,
 'court': 1,
 'either': 3,
 'templ': 

In [ ]:
def _create_idf_matrix(freq_matrix, count_doc_per_words, total_documents):
    idf_matrix = {}

    for sent, f_table in freq_matrix.items():
        idf_table = {}

        for word in f_table.keys():
            idf_table[word] = math.log10(total_documents / float(count_doc_per_words[word]))

        idf_matrix[sent] = idf_table

    return idf_matrix
idf_matrix = _create_idf_matrix(frequency_matrix, documents_per_words, total_documents)
idf_matrix

{'\nIN WHICH PHILEAS FOGG AND PASSEPARTOUT ACCEPT EACH OTHER,\nTHE ONE AS MASTER, THE OTHER AS MAN\n\n\nMr. Phileas Fogg lived, in 1872, at No.': {'in': 1.8388490907372552,
  'which': 1.8388490907372552,
  'philea': 0.7596678446896306,
  'fogg': 0.6084001693589813,
  'and': 1.8388490907372552,
  'passepartout': 0.9937510507229985,
  'accept': 1.8388490907372552,
  'each': 1.8388490907372552,
  'other': 1.8388490907372552,
  'the': 1.0606978403536118,
  'one': 1.1398790864012365,
  'as': 1.8388490907372552,
  'master': 1.5378190950732742,
  'man': 1.3617278360175928,
  'live': 1.1398790864012365,
  'no': 1.5378190950732742},
 '7, Saville Row, Burlington\nGardens, the house in which Sheridan died in 1814.': {'savil': 1.0606978403536118,
  'row': 1.0606978403536118,
  'burlington': 1.8388490907372552,
  'garden': 1.8388490907372552,
  'hous': 1.236789099409293,
  'sheridan': 1.8388490907372552,
  'die': 1.8388490907372552},
 'He was one of the\nmost noticeable members of the Reform Club, 

In [ ]:
def _create_tf_idf_matrix(tf_matrix, idf_matrix):
    tf_idf_matrix = {}

    for (sent1, f_table1), (sent2, f_table2) in zip(tf_matrix.items(), idf_matrix.items()):

        tf_idf_table = {}

        for (word1, value1), (word2, value2) in zip(f_table1.items(),
                                                    f_table2.items()):  # here, keys are the same in both the table
            tf_idf_table[word1] = float(value1 * value2)

        tf_idf_matrix[sent1] = tf_idf_table

    return tf_idf_matrix
tf_idf_matrix = _create_tf_idf_matrix(tf_matrix, idf_matrix)
tf_idf_matrix

{'\nIN WHICH PHILEAS FOGG AND PASSEPARTOUT ACCEPT EACH OTHER,\nTHE ONE AS MASTER, THE OTHER AS MAN\n\n\nMr. Phileas Fogg lived, in 1872, at No.': {'in': 0.11492806817107845,
  'which': 0.11492806817107845,
  'philea': 0.09495848058620382,
  'fogg': 0.07605002116987267,
  'and': 0.11492806817107845,
  'passepartout': 0.06210944067018741,
  'accept': 0.11492806817107845,
  'each': 0.11492806817107845,
  'other': 0.2298561363421569,
  'the': 0.13258723004420148,
  'one': 0.07124244290007728,
  'as': 0.2298561363421569,
  'master': 0.09611369344207964,
  'man': 0.08510798975109955,
  'live': 0.07124244290007728,
  'no': 0.09611369344207964},
 '7, Saville Row, Burlington\nGardens, the house in which Sheridan died in 1814.': {'savil': 0.15152826290765883,
  'row': 0.15152826290765883,
  'burlington': 0.2626927272481793,
  'garden': 0.2626927272481793,
  'hous': 0.1766841570584704,
  'sheridan': 0.2626927272481793,
  'die': 0.2626927272481793},
 'He was one of the\nmost noticeable members of 

In [ ]:
vectorizer = TfidfVectorizer(stop_words="english")
transformed = vectorizer.fit_transform(sentences)
tfidf_transformer=TfidfTransformer(use_idf=True)
tfidf_transformer.fit(transformed)

TfidfTransformer()

In [ ]:
tfidf_transformer.idf_

array([4.55534806, 4.55534806, 4.14988295, 4.55534806, 4.55534806,
       4.55534806, 4.55534806, 4.55534806, 4.14988295, 4.55534806,
       4.14988295, 4.55534806, 4.55534806, 4.55534806, 4.55534806,
       4.55534806, 4.55534806, 4.55534806, 4.55534806, 4.55534806,
       4.55534806, 4.55534806, 4.55534806, 4.55534806, 4.55534806,
       4.55534806, 4.55534806, 4.55534806, 4.55534806, 4.55534806,
       4.55534806, 4.55534806, 4.55534806, 4.55534806, 4.55534806,
       4.55534806, 4.14988295, 4.55534806, 4.55534806, 4.55534806,
       4.55534806, 4.14988295, 4.55534806, 4.55534806, 4.55534806,
       4.55534806, 4.55534806, 4.55534806, 4.14988295, 4.55534806,
       4.14988295, 4.55534806, 4.55534806, 4.55534806, 4.55534806,
       4.55534806, 4.55534806, 4.55534806, 4.55534806, 3.86220088,
       4.55534806, 4.55534806, 4.55534806, 4.55534806, 4.55534806,
       4.55534806, 4.55534806, 4.55534806, 4.55534806, 4.55534806,
       4.55534806, 4.55534806, 4.55534806, 4.55534806, 4.55534

In [ ]:
s_sum={}

for bgn,wf in tf_idf_matrix.items():
    s_sum[bgn] = sum(wf.values())/len(wf)

In [ ]:

suum = sum(s_sum.values())
average_score = suum/len(s_sum)
print("averagge score :",average_score)

averagge score : 0.2536238507265348


In [ ]:
summary=" "
for sentence in sentences:
    if(s_sum[sentence]>=average_score):
        summary +=" "+sentence

summary

'  He was recommended by the Barings, with whom he had an open credit. Was Phileas Fogg rich? Undoubtedly. He was, in short, the least\ncommunicative of men. Had he travelled? Mr. Fogg\nplayed, not to win, but for the sake of playing. A single domestic sufficed to\nserve him. "The new servant," said he. A young man of thirty advanced and bowed. I believe I\'m honest,\nmonsieur, but, to be outspoken, I\'ve had several trades. "Passepartout suits me," responded Mr. Fogg. "You are well recommended\nto me; I hear a good report of you. You know my conditions?" "Yes, monsieur." "Good! What time is it?" "You are too slow," said Mr. Fogg. No matter; it\'s enough to mention the\nerror.'

In [ ]:
import spacy
nlp = spacy.load('en_core_web_sm')
doc1 = nlp(u'/content/gdrive/MyDrive/project/sum.txt')
doc2 = nlp(u'/content/gdrive/MyDrive/project/Tf - idf summary .txt')
n  = (doc1.similarity(doc2))
print (n*100)

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


91.64122586156742


<ipython-input-21-ba811cc93813>:5: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  n  = (doc1.similarity(doc2))


In [ ]:
import spacy
nlp = spacy.load('en_core_web_sm')
doc1 = nlp(u'/content/gdrive/MyDrive/project/sum2.0.txt')
doc2 = nlp(u'/content/gdrive/MyDrive/project/Tf - idf summary .txt')
n  = (doc1.similarity(doc2))
print (n*100)

92.82686450448708


<ipython-input-22-f07b63062805>:5: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  n  = (doc1.similarity(doc2))


In [ ]:
import spacy
nlp = spacy.load('en_core_web_sm')
doc1 = nlp(u'/content/gdrive/MyDrive/project/sum3.0.txt')
doc2 = nlp(u'/content/gdrive/MyDrive/project/Tf - idf summary .txt')
n  = (doc1.similarity(doc2))
print (n*100)

89.35790484316595


<ipython-input-23-cd25b8e06074>:5: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  n  = (doc1.similarity(doc2))
